In [298]:
# Basic SciPy packages
import numpy as np
import pandas as pd

# Stats Packages
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats

# Graphing packages
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
#%matplotlib inline
%matplotlib tk

#Load my random dataset
import CreateDataSet


In [9]:
#Generate Dataset

cvd_dataset = CreateDataSet.make_cvd_dataset()
cvd_dataset

,Power,GrowthRate
0,8,5000.000098
1,8,5012.664928
2,8,5018.486875
3,8,5000.003111
4,10,5050.553024
5,10,5051.092497
6,10,5051.348757
7,10,5080.248275
8,12,5106.268943
9,12,5101.476174


In [299]:
## Save dataset to csv to lock it in. 
#cvd_dataset.to_csv("CVD_Dataset_EDA_ANOVA.csv")
cvd_dataset = pd.read_csv("CVD_Dataset_EDA_ANOVA.csv")

In [336]:
sns.boxplot(x = "Power", y = "GrowthRate", data = cvd_dataset)

In [337]:
cvd_dataset_initial.boxplot('GrowthRate', by ='Power')

In [326]:
## Create my own ANOVA
cvd_dataset_initial = cvd_dataset

In [302]:
runID = pd.Series(data=[1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4],name='RunID')
cvd_dataset['RunID'] = runID
cvd_dataset

,Power,GrowthRate,RunID
0,8,5000.000098,1
1,8,5012.664928,2
2,8,5018.486875,3
3,8,5000.003111,4
4,10,5050.553024,1
5,10,5051.092497,2
6,10,5051.348757,3
7,10,5080.248275,4
8,12,5106.268943,1
9,12,5101.476174,2


In [303]:
cvd_pivot = cvd_dataset.pivot(index = 'Power', columns = 'RunID', values = 'GrowthRate')
cvd_pivot

RunID,1,2,3,4
Power,,,,
8,5000.000098,5012.664928,5018.486875,5000.003111
10,5050.553024,5051.092497,5051.348757,5080.248275
12,5106.268943,5101.476174,5106.513395,5107.977620
14,5150.872442,5152.530164,5159.038556,5150.521528


In [304]:
# Define counts
n = cvd_pivot.count(axis=1)
# Select 1 row
a = cvd_pivot.count()

n = n.iloc[0]
a = a.iloc[0]
TotalN = n*a
TotalN

dft = a - 1
dfe = TotalN - a
dftotal = TotalN-1

In [305]:
#Compute sum under i-th treatment
yi_sum = cvd_pivot.sum(axis = 1)
yi_sum

Power
8     20031.155012
10    20233.242553
12    20422.236133
14    20612.962690
dtype: float64

In [306]:
#Compute average under i-th treatment 
yi_avg = cvd_pivot.mean(axis = 1)
yi_avg

Power
8     5007.788753
10    5058.310638
12    5105.559033
14    5153.240673
dtype: float64

In [307]:
#Compute grand sum
y_sum = yi_sum.sum()
y_sum

81299.59638766023

In [308]:
# Compute grand average
y_avg = yi_avg.mean()
y_avg

5081.2247742287645

In [309]:
cvd_pivot

RunID,1,2,3,4
Power,,,,
8,5000.000098,5012.664928,5018.486875,5000.003111
10,5050.553024,5051.092497,5051.348757,5080.248275
12,5106.268943,5101.476174,5106.513395,5107.977620
14,5150.872442,5152.530164,5159.038556,5150.521528


In [310]:
# Varability between treatments
SSTR = (yi_avg - y_avg)**2
SSTR = SSTR.sum()*n
SSTR

46785.410453411532

In [311]:
# Varability within treatments

SSE = cvd_pivot.sub(yi_avg.values, axis = 0) ** 2
SSE = SSE.sum().sum()
SSE

972.5727188495459

In [312]:
## Total corrected sum of squares

SST = SSE - SSTR
SST

-45812.837734561988

In [313]:
## Mean Square Error

MSE = SSE/(dfe)
MSE

81.04772657079549

In [314]:
## Mean Square Treatments
MSTR = SSTR/(dft)
MSTR

15595.136817803845

In [322]:
#F ratio 
F = MSTR/MSE
F

192.41917667093887

In [316]:
p = stats.f.sf(F, dfe, dft)
p

0.00054615547385569035

In [317]:
eta_sqr = SSTR/SST
eta_sqr

-1.0212292616424374

In [318]:
ome_sqr = (SSTR - (a*MSE))/ (SST+ n)
ome_sqr

-1.0142413963075547

In [330]:
# Something screwy happens here for how the function is grabbing data. 
results = smf.ols('GrowthRate~Power', data=cvd_dataset_initial).fit()
anov = sm.stats.anova_lm(results, typ=2)
print(anov)

                sum_sq    df           F        PR(>F)
Power     46774.595481   1.0  665.906582  3.317629e-13
Residual    983.387691  14.0         NaN           NaN


In [320]:
print("Sum of Squares within = {} and Sum of Squares between = {}.".format(SSE, SSTR))

Sum of Squares within = 972.5727188495459 and Sum of Squares between = 46785.41045341153.


In [321]:
print("F ratio is {} and the p is {}." .format(F,p))

F ratio is 192.41917667093887 and the p is 0.0005461554738556904.


In [324]:
46774/(983.38/14)

665.9033130631088

In [331]:
F, p = stats.f_oneway(cvd_dataset_initial['Power'], cvd_dataset_initial['GrowthRate'])

In [332]:
F, p

(128971.21938580343, 4.5469472829539983e-56)